In [1]:
%matplotlib inline  
%config InlineBackend.figure_format = 'retina'

from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: right;
    vertical-align: middle;
}
</style>
""")

# system ----
import os
import sys
import matplotlib
import h5py
import numpy as np
from scipy import integrate
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.legend_handler import HandlerBase

import warnings
warnings.filterwarnings("ignore",category=FutureWarning)
 
# !!! you will have to edit this to your local computer

# local ----
localPath = os.path.expanduser("~/")
projPath = os.path.expanduser(localPath+"_projects/researchproject_014/")
sys.path.append(projPath+"modules")
import plot_parameters 

# !!! 

***

# Sample Data set

This example dataset consists of the PHAT-ELVIS suite of Milky-way size halos with analytical potential profiles (i.e. the galactic bulge and galactic disk) analogous to the Milky Way

In [2]:
# cosmology
params = {
    'h0'     : 0.6751,
    'omegaM' : 0.3121,
    'omegaL' : 0.6879,
    'omegaB' : 0.0000,
    'omegaR' : 0.0000,
    'boxsize': 1,
    }
h0=params['h0']

# ---- load in dataset
hid=str(988) # specific main halo simulation
# rockstar catalog
cat=np.genfromtxt(projPath+'/outputs/pelvis/halo_catalog_disk_'+hid+'.csv',
                  skip_header=1,delimiter=',')
# merger tree catalog
tree=np.genfromtxt(projPath+'/outputs/pelvis/main_branches_disk_'+hid+'_trimmed.csv',
                   skip_header=1,delimiter=',')

***

# Nomenclature

The target halo of this simulations are Milky Way-mass dark matter halos simulated with a growing potential disk and potential bulge (see https://arxiv.org/abs/1811.12413). The target halo, which we will call the **host halo** is a *isolated* dark matter halo that was simulated. A host halo is typically defined to be a halo that did not form nor currently reside in a larger halo. 

For the contents underneath, check out these slides from http://www.astro.yale.edu/vdbosch/astro610_lecture11.pdf.

### Dark Matter Halos

Dark matter halos are typically defined to be objects that have a density that starkingly contrasts from the background density of the universe, $\rho_{\rm bg}$. For example, the background can be defined by the *critical density*, $\rho_{\rm crit}$, or the *mean matter density*, $\rho_{\rm m} := \Omega_{\rm m}\rho_{\rm crit}$. Typically, halos are defined by the contrast via a overdensity parameter, $\Delta$, giving the halo mass (and radius) as

\begin{align}
M_{\Delta} := \frac{4\pi R_{\Delta}^{3}}{3} (\Delta \times \rho_{\rm bg}) 
\, ,
\end{align}

Note that $R_{\Delta}$ is *not a physical boundary* of a dark matter halo.

Halos are defined by many definitions overdensity througout the literature, so you need to thoroughly read in methodology sections of *each paper* to see how they are defined. The most popular definition we will roll with is the classic *virial overdensity*, $\Delta_{\rm vir}$, with the critical density of the universe, $\rho_{\rm crit}(z)$, which are both redshift evolving. Its analytical form is given in https://arxiv.org/abs/astro-ph/9710107. Thus we have dark matter halos defined as

\begin{align}
    M_{\rm vir} := \frac{4\pi R_{\rm vir}^{3}}{3} \Delta_{\rm vir}(z)\, \rho_{\rm crit}(z) 
    \, .
\end{align}

To model the *physical growth*, instead of the pseudo-evolving growth (from $\Delta_{\rm vir}$ and $\rho_{\rm bg}$), of dark matter halos, we can parameterize the profile by the maximum circular velocity,
\begin{align}
    V_{\rm max} := \mathrm{max}\left[ \sqrt{\frac{GM(<R)}{R}} \right]
    \, ,
\end{align}
and the radius, $R_{\rm max}$, at which $V_{\rm max}$ is obtained. 

### Subhalos and Satellites

**Subhalos** (or substructure) are typically defined to be halos that formed inside and/or are located inside a much larger halo. The satellites for the Milky Way are located inside $R_{\rm vir}(z=0)$ at present day. Satellites are typically defined as subhalos orbiting the Milky Way halo, i.e., subhalos *bound* to the potential of Milky Way. We will focus on halos presently found inside the Milky Way halo.

As a side discussion, halos found outside a host are typically referred to as *halos in the field* or the *halos of the background*. 


### Structure of CDM Halos

A classic result from pure CDM simulations are that a variety of halo mass ranges are characterized by a single dark matter profile, i.e., a *universal* *NFW profile*:

\begin{align}
    \rho(r) = \frac{\rho_{s}}{(r/r_{s})(1+r/r_{s})^{2}}
    \, ,
\end{align}

where $\rho_{s}:=\rho(r_{s})$ and $r_{s}$ are, respectively, the scale density and scale radius and can be thought as free parameters. 

A way of connecting how dense these objects are inside $r_{s}$, *relative* to the size of the halo $R_{\rm vir}$, is called the **concentration** of the:

\begin{align}
    c_{\rm vir} = \frac{r_{s}}{R_{\rm vir}}
    \, .
\end{align}

Once you read through the vast literature, papers typically studied the concentration-mass relation of the halos in the background. However, note that concentration for *subhalos* follow a different trend, since their are dynamical effects of the host halo background "stripping" the subhalo mass profile. 

Typically for NFW, one can relate the $V_{\rm max}$ and $R_{\rm max}$ to the concentration. For example, $r_{s} \simeq 2.163\, R_{\rm max}$. Another way of definining the concentration is with $V_{\rm max}/V_{\rm vir}$, where $V_{\rm vir} := GM_{\rm vir}/R_{\rm vir}$. Ways of defining the concentration will be more apparent when you start to think about the concentration of field halos to substructure.

Everything except these structure quantities are defined in the `Cosmology` class below.

In [3]:
class Cosmology(object):

    def __init__(self,h0=0.704,omegaM=0.2726,omegaL=0.7274,omegaB=0.0456,omegaR=0,boxsize=106.5,**kwargs):
        self.h0 = float(h0)
        self.omegaM = float(omegaM)
        self.omegaL = float(omegaL)
        self.omegaB = float(omegaB)
        self.omegaR = float(omegaR)
        self.boxsize = float(boxsize)

    def scale_factor(self, z):
        return 1./(z + 1.)

    def H(self, z):
        H0 = 100 * self.h0 # km/s/Mpc
        aa = self.scale_factor(z)
        omegaMz = (self.omegaM + self.omegaB) * np.power(1.+z, 3)
        omegaLz = self.omegaL
        omegaRz = self.omegaR * np.power(1.+z,4)
        omegaKz = (1. - self.omegaM - self.omegaL - self.omegaR) * np.power(1.+z,2)
        return H0 * np.sqrt(omegaMz + omegaLz) # km/s/Mpc
 
    def delta_vir(self, z): # The virial overdensity from Bryan & Norman (1998)
        a = self.scale_factor(z)
        Omega = self.omegaM*np.power(1.+z,3) / (self.omegaM*np.power(1.+z,3) + self.omegaL)
        x = Omega - 1.
        return (18.*np.pi**2. + 82.*x - 39.*x**2.)
 
    def rho_crit(self, z):
        a = self.scale_factor(z)
        H = self.H(z)/3.09e+19 # 1/s
        G = 6.67408e-11/1000/1000/1000 # Converson of m^3/kg/s^2 --> km^3/kg/s^2
        return 3. * H**2. /(8*np.pi*G) # kg/km^3
 
    def virial_radius(self, z, Mvir):
        Mvir = np.float64(Mvir) * 1.989e+30 # Conversion of Msol --> kg
        return np.power( 3. * Mvir/self.delta_vir(z)/self.rho_crit(z)/(4.*np.pi), 1./3) * 3.24078e-17 # km --> kpc

    def virial_velocity(self, z, Mvir):
        G = 6.67408e-11/1000/1000/1000 # m^3/kg/s^2 --> km^3/kg/s^2
        radius = self.virial_radius(z, Mvir) * 3.086e+16 # kpc --> km
        Mvir = float(Mvir) * 1.989e+30 # Msol --> kg
        return np.sqrt(G*Mvir/radius)

##  Halo catalog @ $z=0$

What I am doing here is loading all of the halo properties from the $z=0$ catalogs and making my selections for the subhalo populations so that we can follow their history through the merger trees. 

First, we load all of the data...

In [17]:
#z=0 halo catalog
# (0)host_id, (1)tree, (2)id, 
# (3)mvir, (4)rs, (5)rvir, (6)vmax,
# (7)vx, (8)vy, (9)vz, (10)x, (11)y, (12)z,
# vpeak, scale_vpeak

In [4]:
#... load data

# The main branch these halos belong to...
treeid=cat[:,1].astype(int); #print(treeid)

# halo mass and halo radius
mvir=cat[:,3]/h0 # Msol 
rvir=cat[:,5]/h0 # ckpc

# Positions are saved in "comoving" units and are (from historical purposes) normalized by h0.
pos=np.zeros((mvir.shape[0],3)) # 3D cartesian coordinates
pos[:,0]=cat[:,10]; pos[:,1]=cat[:,11]; pos[:,2]=cat[:,12];
pos*=1e3/h0 # ckpc
# Since we are only loading the z=0 data, you wont have to worry about converting to physical units here.

# Velocities are saved in physical units called "peculiar velocities"
vel=np.zeros((mvir.shape[0],3)) # 3D cartesian velocities
vel[:,0]=cat[:,7]; vel[:,1]=cat[:,8]; vel[:,2]=cat[:,9]; # km/s

vmax=cat[:,6]; vpeak=cat[:,13]; vp_scale=cat[:,14]

We that, we make our halo selection. First, we find the host MW halo in the simulations, which should be the most massive dark matter halo. We will only select halos within $R_{\rm vir}$ at $z=0$ and only look at "numerically resolved" dark matter subhalos. These will be subhalos with $M_{\rm vir} \gtrsim 7\times10^{7}\ M_{\odot}$ and $V_{\rm max} > 4.5\ \rm\ km\ s^{-1}$. I wouldn't worry about this cutoff right now; we can follow up on this in a later discussion. 

All we want here is the tree root id of the subhalos for the merger tree catalog, which we define as `nstree_id_`.

In [5]:
# find main halo, i.e. the most massive halo at z=0 in this case
w=np.where(mvir==max(mvir))[0]
htreeid=treeid[w][0]
hmass=mvir[w][0]; hrvir=rvir[w][0]
hpos=pos[w][0]; hvel=vel[w][0]

# satellites @ z=0 (everything resolved inside rvir)
streeid=np.delete(treeid,w,0) # all i'm doing is removing the main halo from the previous dataset
smass=np.delete(mvir,w,0); svmax=np.delete(vmax,w,0);
streeid=np.delete(treeid,w,0)
spos=np.delete(pos,w,0); svel=np.delete(vel,w,0)
vpeak=np.delete(vpeak,w,0); vp_scale=np.delete(vp_scale,w,0)

sep=hpos-spos; rsep=np.sqrt(sep[:,0]**2 + sep[:,1]**2 + sep[:,2]**2)
mask=np.where((rsep/hrvir<1.)&(svmax>4.5)&(smass>7e7))[0]; #print(mask.shape[0])
nstree_id_=streeid[mask] # this allow us to use these results with the merger tree data

## Dynamical Equations

The (physical) **relative position** of the satellite at some redshift in the galactocentric frame (GC) of the host is 

\begin{align}
    \boldsymbol{r}_{\rm rel}(z) := \boldsymbol{r}_{\rm host}(z) - \boldsymbol{r}_{\rm sat}(z)\, ,
\end{align}

where $\boldsymbol{r}(z) := a(z)\boldsymbol{x}(z)$ with $\boldsymbol{x}$ denoted as the **comoving position** inside the cosmological box and $a(z)$ is the **scale factor**. Typically, the halo catalogs (like the ones used here) save the halo positions in comoving coordinates and must be converted to physical when appropriate. If you are wanting to find the pericenter of a satellite (the shortest GC distance along its orbit), you can say $r_{\rm peri} = \mathrm{min}|\boldsymbol{r}_{\rm rel}(z)|$.

Similarly, the relative **peculiar velocity** in the GC frame follows

\begin{align}
    \boldsymbol{v}_{\rm rel,pec}(z) := \boldsymbol{v}_{\rm host}(z) - \boldsymbol{v}_{\rm sat}(z)\, .
\end{align}

The halo catalogs always gives the halo velocity in terms of the peculiar (i.e., physical) velocity. The **total** (physical) velocity considered for our analysis accounts the Hubble flow:

\begin{align}
    \boldsymbol{v}_{\rm rel}(z)= H(z)\boldsymbol{r}_{\rm rel}(z) + \underbrace{a(z)\dot{\boldsymbol{x}}_{\rm rel}(z)}_{\boldsymbol{v}_{\rm rel,pec}}\, ,
\end{align}

where $H(z) := \dot{a}/a$. Motions of the satellite halos can be decomposed into simplified dynamical information, one being the **radial velocity**:

\begin{align}
    v_{\rm rad} := \frac{\boldsymbol{r}_{\rm rel}\cdot\boldsymbol{v}_{\rm rel}}{|\boldsymbol{r}_{\rm rel}|}\, ,
\end{align}

while the **tangential velocity** is just $v_{\rm tan}^{2} = v_{\rm rel}^{2} -v_{\rm rad}^{2} $. 


I saved of this information within the `Dynamics` class in the next cell.

In [6]:
class Dynamics(Cosmology):

    def __init__(self, **kwargs):
        super(Dynamics, self).__init__(**kwargs)

    def periodic(self,z,sep): # not used here
        a = self.scale_factor(z)
        boxl = self.boxsize*a*1000 # phyical kpc
        sep[sep < -boxl/2] += boxl
        sep[sep > boxl/2] -= boxl
        return sep

    def distance(self,z,hostpos,satpos):
        results={}
        #a = self.scale_factor(z)
        sep_vec=np.zeros(satpos.shape) # physical kpc
        sep_vec[:,0]=(hostpos[:,0]-satpos[:,0])
        sep_vec[:,1]=(hostpos[:,1]-satpos[:,1])
        sep_vec[:,2]=(hostpos[:,2]-satpos[:,2])
        sep_mag=np.sqrt(np.power(sep_vec[:,0],2)+np.power(sep_vec[:,1],2)+np.power(sep_vec[:,2],2))
        results['relative.distance:vector']=sep_vec
        results['relative.distance:magnitude']=sep_mag
        return results

    def total_velocity(self, z, hostvel, satvel, hostpos, satpos):
        results={}
        #a = self.scale_factor(z)
        H=self.H(z)*1e-3 # km/s/kpc

        dist_dict=self.distance(z,hostpos,satpos) # physical kpc
        reldis_vec=dist_dict['relative.distance:vector']
        reldis_mag=dist_dict['relative.distance:magnitude']

        relvel_vec=np.zeros(satvel.shape) # km/s
        relvel_vec[:,0]=(hostvel[:,0]-satvel[:,0])
        relvel_vec[:,1]=(hostvel[:,1]-satvel[:,1])
        relvel_vec[:,2]=(hostvel[:,2]-satvel[:,2])

        hubble=np.zeros(reldis_vec.shape)
        for i in range(reldis_vec.shape[0]):
            hubble[i]+=reldis_vec[i]*H[i]
        totvel_vec=hubble+relvel_vec # km/s
        #totvel_vec=relvel_vec # km/s
        totvel_mag=np.sqrt(np.power(totvel_vec[:,0],2)+np.power(totvel_vec[:,1],2)+np.power(totvel_vec[:,2],2))

        results['total.velocity:vector']=totvel_vec
        results['total.velocity:magnitude']=totvel_mag
        return results

    def radial_velocity(self, z, hostvel, satvel, hostpos, satpos):
        H=self.H(z)/1000. # km/s/kpc
        #a=self.scale_factor(z)

        dist_dict=self.distance(z,hostpos,satpos) # physical kpc
        reldis_vec=dist_dict['relative.distance:vector']
        reldis_mag=dist_dict['relative.distance:magnitude']
        totvel_dict=self.total_velocity(z,hostvel,satvel,hostpos,satpos) # km/s
        #relvel_vec=hostvel-satvel
        totvel_vec=totvel_dict['total.velocity:vector']

        numer=totvel_vec[:,0]*reldis_vec[:,0]+totvel_vec[:,1]*reldis_vec[:,1]+totvel_vec[:,2]*reldis_vec[:,2]
        return numer/reldis_mag

    def tangential_velocity(self, z, hostvel, satvel, hostpos, satpos):
        rad_vel=self.radial_velocity(z, hostvel, satvel, hostpos, satpos)
        tot_vel=self.total_velocity(z, hostvel, satvel, hostpos, satpos)['total.velocity:magnitude']
        return np.sqrt(np.power(tot_vel,2) - np.power(rad_vel,2))

    def space_velocity(self, z, hostvel, satvel, hostpos, satpos):
        rad_vel=self.radial_velocity(z,hostvel,satvel,hostpos,satpos)
        tot_vel=self.total_velocity(z,hostvel,satvel,hostpos,satpos)['total.velocity:magnitude']
        return tot_vel*np.sign(rad_vel)

## Merger tree data

In [7]:
'''
merger tree info:
        (0) host_id : simulation number. Host mass decreases as this increases
        (1) tree : tree number (main branch) this halo belongs to
        (2) scale : scale factor of the halo properties
        (3) id : specific halo number (unique within `host_id` groups)
        (4) pid : ID of the parent halo (-1 if not a subhalo)
        (5) upids : ID of the uppermost halo (-1 if not a sub-subhalo, equals `pid` if subhalo)
        (6) phantom : Nonzero if the halo is not found by Rockstar, but used to link snapshots
        (7) mass : mass of the subhalo (Msun/h)
        (8) rvir : virial radius (kpc/h comoving)
        (9) rs : scale radius of best fit NFW profile as determined by Rockstar (kpc/h comoving)
        (10)vmax : maximum circular velocity (km/s)
        (11)x : x coordinate of center (Mpc/h comoving)
        (12)y : y coordinate of center (Mpc/h comoving)
        (13)z : z coordinate of center (Mpc/h comoving)
        (14)vx : x bulk velocity (km/s)
        (15)vy : y bulk velocity (km/s)
        (16)vz : z bulk velocity (km/s)
'''

'\nmerger tree info:\n        (0) host_id : simulation number. Host mass decreases as this increases\n        (1) tree : tree number (main branch) this halo belongs to\n        (2) scale : scale factor of the halo properties\n        (3) id : specific halo number (unique within `host_id` groups)\n        (4) pid : ID of the parent halo (-1 if not a subhalo)\n        (5) upids : ID of the uppermost halo (-1 if not a sub-subhalo, equals `pid` if subhalo)\n        (6) phantom : Nonzero if the halo is not found by Rockstar, but used to link snapshots\n        (7) mass : mass of the subhalo (Msun/h)\n        (8) rvir : virial radius (kpc/h comoving)\n        (9) rs : scale radius of best fit NFW profile as determined by Rockstar (kpc/h comoving)\n        (10)vmax : maximum circular velocity (km/s)\n        (11)x : x coordinate of center (Mpc/h comoving)\n        (12)y : y coordinate of center (Mpc/h comoving)\n        (13)z : z coordinate of center (Mpc/h comoving)\n        (14)vx : x bulk 

In [8]:
tree_ids=tree[:,1]; scale=tree[:,2]; redshift=1./scale - 1.
tmvir=tree[:,7]/h0; # Msol
trvir=tree[:,8]/h0 # ckpc
tvmax=tree[:,10]; # ckpc
trmax=tree[:,9]*2.163/h0 # ckpc
xpos=tree[:,11]; ypos=tree[:,12]; zpos=tree[:,13]
xvel=tree[:,14]; yvel=tree[:,15]; zvel=tree[:,16]

Save the **main halo** tree to a dictionary called `host_tree`

In [9]:
host_tree={}
wh=np.where(tree_ids==htreeid)[0]
host_tree['virial.mass']=tmvir[wh] # Msol
host_tree['scale']=scale[wh]; host_tree['redshift']=redshift[wh]
host_tree['virial.radius']= host_tree['scale']*trvir[wh]/h0 # physical kpc

host_tree['position']=np.zeros((host_tree['scale'].shape[0],3)) # physical kpc
host_tree['position'][:,0]=host_tree['scale']*xpos[wh]*1e3/h0
host_tree['position'][:,1]=host_tree['scale']*ypos[wh]*1e3/h0
host_tree['position'][:,2]=host_tree['scale']*zpos[wh]*1e3/h0

host_tree['velocity']=np.zeros((host_tree['scale'].shape[0],3)) # peculiar km/s
host_tree['velocity'][:,0]=xvel[wh]; host_tree['velocity'][:,1]=yvel[wh]; host_tree['velocity'][:,2]=zvel[wh]

Now to the **satellite halo** data that we have saved in `nstree_id_`. We are going to update it by swifting out objects called "orphan" halos. These are halos that sometimes appear out of nowhere from the tree and fast flybys for the host. Objects like these are not typical in zoom-in simulations, but are apparent in large-box cosmologcial simulations. 

To find these objects, we'll loop through each satellite and throw out any that do not have main trees going beyond redshift $z=3$. This will update to an array called `nstree_id`.

In [10]:
#throw out orphan subhalos
nstree_id=[]
for tid in nstree_id_:
    w=np.where(tid==tree_ids)[0]; z=redshift[w]
    if(z[-1]>3):
        nstree_id.append(tid)
    else:
        pass
nstree_id=np.array(nstree_id)

I'll do the whole loop for each satellite, but I'll leave it up to you how you want to save each and every dataset. I'll save each halo with the tree id to a dictionary called `sat_data`.

In [12]:
cosmo=Cosmology(**params)
dyn=Dynamics(**params)

sat_data={} # contains the entire satellite tree data
ftree_id=[] # corrected satellite IDs
for tid in nstree_id[:]:
    sat_tree={}
    
    #partition out satellite quantities baed on tree depth
    ws=np.where(tid==tree_ids)[0];
    sat_tree['virial.mass']=tmvir[ws]
    sat_tree['scale']=scale[ws]; sat_tree['redshift']=redshift[ws]
    sat_tree['vel.max']=tvmax[ws] # physical kpc
    sat_tree['rad.max']=sat_tree['scale']*trmax[ws] # physical kpc
    
    sat_tree['position']=np.zeros((sat_tree['scale'].shape[0],3)) # physical kpc
    sat_tree['position'][:,0]=sat_tree['scale']*xpos[ws]*1e3/h0
    sat_tree['position'][:,1]=sat_tree['scale']*ypos[ws]*1e3/h0
    sat_tree['position'][:,2]=sat_tree['scale']*zpos[ws]*1e3/h0
    
    sat_tree['velocity']=np.zeros((sat_tree['scale'].shape[0],3)) # peculiar km/s
    sat_tree['velocity'][:,0]=xvel[ws]; sat_tree['velocity'][:,1]=yvel[ws]; sat_tree['velocity'][:,2]=zvel[ws]

    '''Sometimes the satellite halos will have missing tree data at higher redshifts compared to MW, 
    which we expect to 'complete'. Below is a routine to make sure the host halo data and the satellite data 
    match at each snapshot/redshift.'''
    
    #partition out matching redshift host halo quantities
    wh=np.where(np.in1d(host_tree['scale'],sat_tree['scale']))[0]
    if(wh.shape[0]!=sat_tree['scale'].shape[0]):
        # I was too lazy to correct this...
        pass
    else:
        ftree_id.append(int(tid))
        sat_data[str(int(tid))]=sat_tree
            
        # matched snapshot host values
        # use these when doing dynamical analysis
        sat_tree['distance']=dyn.distance(sat_tree['redshift'],
                                          host_tree['position'][wh],sat_tree['position'])['relative.distance:magnitude']
        sat_tree['distance:pericenter']=min(sat_tree['distance'])
        
        sat_tree['total.velocity']=dyn.total_velocity(sat_tree['redshift'],host_tree['velocity'][wh],sat_tree['velocity'],host_tree['position'][wh],sat_tree['position'])['total.velocity:magnitude']
        sat_tree['radial.velocity']=dyn.radial_velocity(sat_tree['redshift'],host_tree['velocity'][wh],sat_tree['velocity'],host_tree['position'][wh],sat_tree['position'])
        sat_tree['tangential.velocity']=dyn.tangential_velocity(sat_tree['redshift'],host_tree['velocity'][wh],sat_tree['velocity'],host_tree['position'][wh],sat_tree['position'])
        sat_tree['space.velocity']=dyn.space_velocity(sat_tree['redshift'],host_tree['velocity'][wh],sat_tree['velocity'],host_tree['position'][wh],sat_tree['position'])

In [16]:
print(sat_data['25258073']['distance'])
#print(sat_data['25258073']['radial.velocity'])

[116.70413868 116.94377791 115.6545454  108.09440424  97.93867081
  94.0932472  101.7932344  109.57140021 110.08992235 103.05019895
  88.95259797  36.15744269  37.65630867  48.59170508  61.80393139
  75.11393642  85.73744346  95.37390869 104.28609225 110.02703714
  92.15296256 118.63315353 120.89386739 121.66118483 120.36996121
 117.32863248 112.70484102 105.39338885  96.67369544  86.66733465
  74.28887292  61.51001424  49.78684759  43.45520491  52.3090255
  58.03255239  67.93085066  77.85073225  86.74999344  94.28693437
 100.39769548 105.07546441 108.76040766 111.4495865  113.31611128
 114.44691042 115.34506403 116.62242268 117.47419782 116.00019957
 112.01491028 106.43708567  99.46383063  91.06730645  81.7124946
  71.89294866  62.45089233  55.61805174  54.8665906   61.78631087
  72.54713144  83.57351403  93.84467018 103.87939714 113.63790878
 123.52888733 132.9871353  141.93637628 152.53188823 164.66092754
 177.65539181 190.82791196 203.6144766  215.45027882 225.80849646
 234.6780008